In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('../data/final_df.csv')

/var/folders/kb/3pjtmdb110z0myp6wf86bq0h0000gn/T/ipykernel_35793/1140776904.py:1: DtypeWarning: Columns (5,6,9,10,12,13,14,15,16,17,18,19,21,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/final_df.csv')


In [33]:
df = df[df['Aftertreatment System'].notna()]
df = df.drop(df.loc[:,'AcceleratorPedal':'TurboBoostPressure'], axis = 1)
df = df.drop(['EventTimeStamp', 'ESS_Id', 'fmi'], axis = 1)

In [34]:
#Get list of unique EquipmentIDs
eq_ids = df.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1026

In [35]:
import numpy as np

#create a column of random numbers from 0-1
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

eq_ids

,EquipmentID,random
0,301,0.6682
412,302,0.4220
2011,303,0.5929
2738,304,0.9617
3952,305,0.8360
...,...,...
545690,2371,0.1721
545693,2374,0.3518
545704,2375,0.0887
545708,2376,0.5903


In [36]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [37]:
#merge modeling column onto original dataframe
final_df = df.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

final_df = df.merge(eq_ids, on='EquipmentID')

In [38]:
#created training dataframe
df_training = final_df[final_df['modeling']=='training']
df_training = df_training.drop(columns=['modeling', 'random'])
df_training

,EquipmentID,spn,Aftertreatment System,Ambient Air Temperature,Anti-Lock Braking (ABS) Controller,Auxiliary Input/Output,Auxiliary Temperature Sensor Input Circuit,Barometric Pressure Sensor,Battery,Crankcase,...,Sensor Supply,Steering Wheel Angle Sensor,Total Vehicle Distance,Transmission Communications Failure Idle Torque Limit,Transmission Oil,Turbocharger,Unknown,Variable Geometry Turbocharger,Vehicle Identification Number,Wheel-Based Vehicle Speed
6490,306,596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6491,306,596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6492,306,596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
6493,306,596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6494,306,596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545692,2371,111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
545704,2375,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
545705,2375,2863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
545706,2375,2863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [39]:
#create dataframe of both training and testing to have large test dataset for testing.
#With size of dataset, probably can just keep it confined to modeling == 'testing'
df_train_test = final_df[final_df['modeling']!='validation']
df_train_test = df_train_test.drop(columns=['modeling', 'random'])
df_train_test.shape

(397175, 57)

In [40]:
#final dataframe without modeling or random column for final predictions
df_total = final_df.drop(columns=['modeling', 'random'])

In [41]:
#setting up for model testing with training dataset. Realized here about how may need to adjust target variable
#to implement oversampling
features = df_total.iloc[:,2:].columns.tolist()

target =['spn']

X_train = df_training[features]
y_train = df_training[target]

In [42]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

In [43]:
logreg.fit(X_train, y_train)

/Users/bryanfinlayson/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/bryanfinlayson/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [44]:
df_test = final_df[final_df['modeling']=='testing']
df_test = df_test.drop(columns=['modeling','random'])
X_test = df_test[features]
y_test = df_test[target]

In [45]:
from sklearn.metrics import accuracy_score, f1_score

y_pred = logreg.predict(X_test)

print(accuracy_score(y_test,y_pred))
print(f1_score(y_test, y_pred, average = 'weighted'))

0.6139616493294899
0.5196129765371617


In [47]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print(accuracy_score(y_test,y_pred))
print(f1_score(y_test, y_pred, average = 'weighted'))

/var/folders/kb/3pjtmdb110z0myp6wf86bq0h0000gn/T/ipykernel_35793/2504461268.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


0.6375786439403434
0.6104621821561477


In [48]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

print(accuracy_score(y_test,y_pred))
print(f1_score(y_test, y_pred, average = 'weighted'))

0.5672690813385136
0.5656826423505755
